# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [8]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

# for sqrt_value in generator:
#     print(sqrt_value)

sum_5 = sum([x for x in generator])
sum_5

limit = 13
generator = square_root_generator(limit)
for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [27]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

peoples = conn.sql("SELECT * FROM people").df()
display(peoples)





Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707155341.059961,cM0tTk8oLG0e1w,None
1,2,Person_2,27,City_A,1707155341.059961,J98SPz3UKJ2+ag,None
2,3,Person_3,28,City_A,1707155341.059961,oEmc0cf8wSkcnQ,None
3,4,Person_4,29,City_A,1707155341.059961,C8GKeW1HtbLgnA,None
4,5,Person_5,30,City_A,1707155341.059961,dDJpI2OrvGbu/A,None


In [28]:
sum_peoples = peoples["age"].sum()
sum_peoples

140

In [29]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


info_2 = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

peoples = conn.sql("SELECT * FROM people").df()
display(peoples)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707155341.059961,cM0tTk8oLG0e1w,None
1,2,Person_2,27,City_A,1707155341.059961,J98SPz3UKJ2+ag,None
2,3,Person_3,28,City_A,1707155341.059961,oEmc0cf8wSkcnQ,None
3,4,Person_4,29,City_A,1707155341.059961,C8GKeW1HtbLgnA,None
4,5,Person_5,30,City_A,1707155341.059961,dDJpI2OrvGbu/A,None
5,3,Person_3,33,City_B,1707155348.9274082,V6eOGqOCz4e77w,Job_3
6,4,Person_4,34,City_B,1707155348.9274082,gkzLtP2KNgHt7Q,Job_4
7,5,Person_5,35,City_B,1707155348.9274082,k97WtaTA7yWABw,Job_5
8,6,Person_6,36,City_B,1707155348.9274082,Ls15ZCaP5+Q4Wg,Job_6
9,7,Person_7,37,City_B,1707155348.9274082,996YefaDGrG3xQ,Job_7


In [30]:
sum_peoples = peoples["age"].sum()
sum_peoples

353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [36]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
info = generators_pipeline.run(people_1(),
										table_name="people_2",
										write_disposition="merge")


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

peoples = conn.sql("SELECT * FROM people_2").df()
display(peoples)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_2            │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707155680.51563,RtYucTD74d+BMQ,None
1,2,Person_2,27,City_A,1707155680.51563,yrF3W8Oiz8mupQ,None
2,3,Person_3,28,City_A,1707155680.51563,g6ZrlsjneuFkcw,None
3,4,Person_4,29,City_A,1707155680.51563,SWuRYpPDkK+Y2w,None
4,5,Person_5,30,City_A,1707155680.51563,hm7IMPqNGpl7WQ,None


In [37]:
info_2b = generators_pipeline.run(people_2(),
										table_name="people_2",
										write_disposition="replace")


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

peoples = conn.sql("SELECT * FROM people_2").df()
display(peoples)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_2            │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,3,Person_3,33,City_B,1707155692.7843509,m6a1j9LPA69YgA,Job_3
1,4,Person_4,34,City_B,1707155692.7843509,55jfAYdA4eSBJQ,Job_4
2,5,Person_5,35,City_B,1707155692.7843509,AL1c3A85uTpKRQ,Job_5
3,6,Person_6,36,City_B,1707155692.7843509,RiJK03z9Z9J1Xw,Job_6
4,7,Person_7,37,City_B,1707155692.7843509,VvG38o1yFrzRUw,Job_7
5,8,Person_8,38,City_B,1707155692.7843509,NZoqIKNJblwK6g,Job_8


In [38]:
sum_peoples_2 = peoples["age"].sum()
sum_peoples_2

213

# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX